In [1]:
import os

In [2]:
%pwd

'h:\\Winequality-Predcition\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'h:\\Winequality-Predcition'

## Data Validation 

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [6]:

from src.wine_quality.constants import *
from src.wine_quality.utils.common import read_yaml,create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_validation_config(self) -> DataValidationConfig:
        config1 = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config1.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config1.root_dir,
            STATUS_FILE=config1.STATUS_FILE,
            unzip_data_dir = config1.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config

In [8]:
import pandas as pd

class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self) -> bool:
        try:
            data = pd.read_csv(self.config.unzip_data_dir)
            all_columns = set(data.columns)
            all_schema = self.config.all_schema

            validation_status = True

            # Use self.config.STATUS_FILE instead of self.status
            with open(self.config.STATUS_FILE, "w") as file:
                for col, dtype in all_schema.items():
                    if col not in all_columns:
                        validation_status = False
                        file.write(f"Column '{col}' is missing.\n")
                    elif str(data[col].dtype) != dtype:
                        validation_status = False
                        file.write(f"Column '{col}' has incorrect dtype. Expected: {dtype}, Found: {str(data[col].dtype)}\n")
                    else:
                        file.write(f"Column '{col}' dtype matches the expected dtype.\n")

                file.write(f"Data Validation status : {validation_status}\n")

            return validation_status

        except Exception as e:
            raise e


### Pipeline

In [9]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(data_validation_config)
    data_validation.validate_all_columns()
    
except Exception as e:
    raise e

[2024-09-22 11:49:35,133 : INFO : common : yaml file: config\config.yaml loaded successfully ]
[2024-09-22 11:49:35,138 : INFO : common : yaml file: params.yaml loaded successfully ]
[2024-09-22 11:49:35,142 : INFO : common : yaml file: schema.yaml loaded successfully ]
[2024-09-22 11:49:35,144 : INFO : common : created directory at: artifacts ]
[2024-09-22 11:49:35,146 : INFO : common : created directory at: artifacts/data_validation ]
